In [1]:
import numpy as np 
import open3d as o3d
import time

Zero_time = time.perf_counter() 

In [2]:
print("Import meshes ...")
mesh = o3d.io.read_triangle_mesh(r"C:\Data\Temp\012103-40-BIM-Gent\BIM\OBJ\floor.obj")
print(mesh)
t1 = time.perf_counter()
dt1 = t1 - Zero_time
print("Meshes loaded in %s s" % dt1)
print(mesh)
print("Convert Mesh to pointcloud")
meshpcd = mesh.sample_points_uniformly(number_of_points = 10000000, use_triangle_normal=True)
t2 = time.perf_counter()
dt2 = t2 - t1
print("Created reference cloud from mesh in %s s" % dt2)
print(meshpcd)

print("Load targeted pointcloudpoint cloud ...")
pcd = o3d.io.read_point_cloud(r"C:\Data\Temp\012103-40-BIM-Gent\RESULTS\Annotations\Full.pcd")
t3 = time.perf_counter()
dt3 = t3 - t2
print("Target cloud loaded in %s s" % dt3)
print(pcd)

Import meshes ...
TriangleMesh with 3633 points and 1211 triangles.
Meshes loaded in 1.7024483999999998 s
TriangleMesh with 3633 points and 1211 triangles.
Convert Mesh to pointcloud
Created reference cloud from mesh in 0.6856942999999998 s
PointCloud with 10000000 points.
Load targeted pointcloudpoint cloud ...
Target cloud loaded in 1.4152816000000001 s
PointCloud with 7759086 points.


In [3]:
print(pcd.has_normals())
print(meshpcd.has_normals())

True
True


In [4]:
o3d.visualization.draw_geometries([mesh,pcd])
o3d.visualization.draw_geometries([meshpcd])

In [5]:
voxel_size = 0.02

print("Downsampling Pointclouds to excalerate computations Voxel size: %s m" % voxel_size)

ref_voxel = meshpcd.voxel_down_sample(voxel_size)
t4 = time.perf_counter()
dt4 = t4 - t3
print("Downsampled the reference pointcloud in %s s" % dt4)
print(ref_voxel)

tar_voxel = pcd.voxel_down_sample(voxel_size)

t5 = time.perf_counter()
dt5 = t5 - t4
print("Downsampled the target pointcloud in %s s" % dt5)
print(tar_voxel)


Downsampling Pointclouds to excalerate computations Voxel size: 0.02 m
Downsampled the reference pointcloud in 25.3944324 s
PointCloud with 1171494 points.
Downsampled the target pointcloud in 2.5462767 s
PointCloud with 3208779 points.


In [6]:
o3d.visualization.draw_geometries([ref_voxel])
o3d.visualization.draw_geometries([tar_voxel])

In [7]:
print(ref_voxel.has_normals())
print(tar_voxel.has_normals())

True
True


In [8]:
c2c = tar_voxel.compute_point_cloud_distance(ref_voxel)
t6 = time.perf_counter()
dt6 = t6 - t5
c2cArray = np.asarray(c2c)
print("Cloud 2 cloud calculated in %s s" % dt6)
print(c2cArray)

Cloud 2 cloud calculated in 74.8106223 s
[0.06808726 0.00626655 0.01065923 ... 4.53221826 4.53262253 4.56550146]


In [9]:
tar_voxel.paint_uniform_color([0.5,0.5,0.5]) #Grey

PointCloud with 3208779 points.

In [10]:
o3d.visualization.draw_geometries([tar_voxel])

In [11]:
i =0
Inlier = []
Outlier = []
Outlier_points = []
Inlier_points = []
Inlier_normals = []
while i < len(c2c):
    if c2c[i] <= 0.1:
        np.asarray(tar_voxel.colors)[i] = [0,1,0] #Green
        Inlier.append(np.append(tar_voxel.points[i],c2c[i]))
        Inlier_points.append(tar_voxel.points[i])
        Inlier_normals.append(tar_voxel.normals[i])
    elif c2c[i] > 0.1:
        Outlier.append(np.append(tar_voxel.points[i],c2c[i]))
        Outlier_points.append(tar_voxel.points[i])
        np.asarray(tar_voxel.colors)[i] = [1,0,0] #Red
    i = i+1

t7 = time.perf_counter()
dt7 = t7 - t6
c2cArray = np.asarray(c2c)
print("Seperated in %s s" % dt7)

Seperated in 100.731227 s


In [12]:
Inlier_pcd = o3d.geometry.PointCloud()
Inlier_pcd.points = o3d.utility.Vector3dVector(Inlier_points)
Inlier_pcd.normals = o3d.utility.Vector3dVector(Inlier_normals)
print(np.asarray(Inlier_pcd))
o3d.visualization.draw_geometries([Inlier_pcd])

PointCloud with 862745 points.


In [13]:
print(Inlier_pcd.has_normals())

True


In [14]:
print(Inlier_normals[0:10])

[array([-0.59830534,  0.00942397, -0.32389865]), array([0.07467524, 0.01060498, 0.97862428]), array([0.31351587, 0.78141734, 0.53115213]), array([ 0.0233843 , -0.01947222, -0.99941528]), array([-0.13186828, -0.10826411,  0.98482838]), array([-0.05056142, -0.01564641,  0.99430597]), array([0.12273337, 0.01883892, 0.98879145]), array([-0.0069608 ,  0.0627928 , -0.99539779]), array([-0.35972089,  0.91834629, -0.16504841]), array([-0.00573967,  0.00885221,  0.99838771])]


In [21]:
Inlier_pcd.paint_uniform_color([0.5,0.5,0.5]) #Grey

PointCloud with 862745 points.

De Dichtste mesh vinden 

Normaal van het punt vergelijken met de normaal van de mesh

Afwijkende normale ditchen 


In [16]:
#Normalen die niet bij een verticale hoeren weg smijten
i =0
Floor = []
Ceiling = []
Outlier = []
Outlier_points = []
Floor_points = []
Ceiling_points = []
Floor_normals = []
Ceiling_normals = []
while i < len(np.asarray(Inlier_pcd.normals)):
    normals = np.asarray(Inlier_pcd.normals)[i]
    if normals[2] > 0.9:
        np.asarray(Inlier_pcd.colors)[i] = [0,1,0] #Green
        Floor.append(Inlier_pcd.points[i])
        Floor_points.append(Inlier_pcd.points[i])
        Floor_normals.append(Inlier_pcd.normals[i])
    elif normals[2] < -0.9:
        np.asarray(Inlier_pcd.colors)[i] = [0,0,1] #Blue
        Ceiling.append(Inlier_pcd.points[i])
        Ceiling_points.append(Inlier_pcd.points[i])
        Ceiling_normals.append(Inlier_pcd.normals[i])
    else:
        np.asarray(Inlier_pcd.colors)[i] = [1,0,0] #Red
    i = i+1
t8 = time.perf_counter()
dt8 = t8 - t7
print("Seperated in %s s" % dt8)

Seperated in 44.26110370000001 s


In [20]:
i =0
Floor = []
Ceiling = []
Outlier = []
Outlier_points = []
Floor_points = []
Ceiling_points = []
Floor_normals = []
Ceiling_normals = []
while i < len(np.asarray(Inlier_pcd.points)):
    normals = np.asarray(Inlier_pcd.normals)[i]
    if normals[2] > 0.9:
        np.asarray(Inlier_pcd.colors)[i] = [0,1,0] #Green
        Floor.append(Inlier_pcd.points[i])
        Floor_points.append(Inlier_pcd.points[i])
        Floor_normals.append(Inlier_pcd.normals[i])
    elif normals[2] < -0.9:
        np.asarray(Inlier_pcd.colors)[i] = [0,0,1] #Blue
        Ceiling.append(Inlier_pcd.points[i])
        Ceiling_points.append(Inlier_pcd.points[i])
        Ceiling_normals.append(Inlier_pcd.normals[i])
    else:
        np.asarray(Inlier_pcd.colors)[i] = [1,0,0] #Red
    i = i+1
t8 = time.perf_counter()
dt8 = t8 - t7
print("Seperated in %s s" % dt8)

Seperated in 1129.9390482000001 s


In [ ]:
Inlier_pcd.colors[200] = [1, 0, 0]# Given query points and rendered in red
print(meshpcd)
meshpcd_tree = o3d.geometry.KDTreeFlann(pcd)#Create KD tree index
[k, idx, _] = pcd_tree.search_knn_vector_3d(Inlier_pcd.points[200], 200)#K Nearest Neighbor Search
o3d.np.asarray(Inlier_pcd.colors)[idx[1:], :] = [0, 1, 0]#K neighborhood points, rendered in green

In [17]:
o3d.visualization.draw_geometries([Inlier_pcd])

In [18]:
o3d.io.write_point_cloud("Cloud.pcd", Inlier_pcd, print_progress=True)

True